In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.utils import to_categorical
from keras.preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [2]:
#Load dataset into dataframe
data = pd.read_csv('data/fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
#Split training and testing data
train = data[data['Usage'] == 'Training']
test = data[data['Usage'] != 'Training']

In [4]:
train.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
test.head()

,emotion,pixels,Usage
28709,0,254 254 254 254 254 249 255 160 2 58 53 70 77 ...,PublicTest
28710,1,156 184 198 202 204 207 210 212 213 214 215 21...,PublicTest
28711,4,69 118 61 60 96 121 103 87 103 88 70 90 115 12...,PublicTest
28712,6,205 203 236 157 83 158 120 116 94 86 155 180 2...,PublicTest
28713,3,87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...,PublicTest


In [6]:
#Function to prepare data for model
def prepare_data(data) :
    y_data = keras.utils.to_categorical(data['emotion'])
    X_data = np.zeros(shape=(len(data), 48, 48, 1))

    for i, row in enumerate(data.index):
        pixels = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        pixels = np.reshape(pixels, (48, 48)) 
        X_data[i, :, :, 0] = pixels / 255

    return X_data, y_data

In [7]:
#Prepare data
X_train, y_train = prepare_data(train)
X_test, y_test = prepare_data(test)

print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

[[[0.2745098 ]
  [0.31372549]
  [0.32156863]
  ...
  [0.20392157]
  [0.16862745]
  [0.16078431]]

 [[0.25490196]
  [0.23921569]
  [0.22745098]
  ...
  [0.21960784]
  [0.20392157]
  [0.17254902]]

 [[0.19607843]
  [0.16862745]
  [0.21176471]
  ...
  [0.19215686]
  [0.21960784]
  [0.18431373]]

 ...

 [[0.35686275]
  [0.25490196]
  [0.16470588]
  ...
  [0.28235294]
  [0.21960784]
  [0.16862745]]

 [[0.30196078]
  [0.32156863]
  [0.30980392]
  ...
  [0.41176471]
  [0.2745098 ]
  [0.18039216]]

 [[0.30196078]
  [0.28235294]
  [0.32941176]
  ...
  [0.41568627]
  [0.42745098]
  [0.32156863]]]
[1. 0. 0. 0. 0. 0. 0.]
[[[0.99607843]
  [0.99607843]
  [0.99607843]
  ...
  [0.0627451 ]
  [0.        ]
  [0.63137255]]

 [[0.99607843]
  [0.99607843]
  [0.99607843]
  ...
  [0.06666667]
  [0.        ]
  [0.47843137]]

 [[0.99607843]
  [0.99607843]
  [0.99607843]
  ...
  [0.10196078]
  [0.        ]
  [0.44705882]]

 ...

 [[0.25882353]
  [0.3372549 ]
  [0.38823529]
  ...
  [1.        ]
  [0.98823529]
  

In [8]:
#Create CNN model
cnn_model = Sequential()

cnn_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())

cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.5))

cnn_model.add(Dense(7, activation='softmax'))

cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0

In [9]:
#Set optimizer and loss function
opt = keras.optimizers.Adam(lr=0.001)
cnn_model.compile(loss='categorical_crossentropy',
                  optimizer=opt, metrics=['accuracy'])

In [10]:
#Train model for 25 epochs with batch size of 128 items

h1 = cnn_model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1, 
                   validation_data =(X_test, y_test))

Epoch 1/20
225/225 [==============================] - 83s 354ms/step - loss: 1.7946 - accuracy: 0.2694 - val_loss: 1.5807 - val_accuracy: 0.3966
Epoch 2/20
225/225 [==============================] - 54s 239ms/step - loss: 1.5826 - accuracy: 0.3856 - val_loss: 1.4773 - val_accuracy: 0.4418
Epoch 3/20
225/225 [==============================] - 54s 239ms/step - loss: 1.4722 - accuracy: 0.4320 - val_loss: 1.3926 - val_accuracy: 0.4575
Epoch 4/20
225/225 [==============================] - 54s 238ms/step - loss: 1.4020 - accuracy: 0.4650 - val_loss: 1.3310 - val_accuracy: 0.4883
Epoch 5/20
225/225 [==============================] - 54s 239ms/step - loss: 1.3557 - accuracy: 0.4813 - val_loss: 1.2948 - val_accuracy: 0.5049
Epoch 6/20
225/225 [==============================] - 57s 254ms/step - loss: 1.3174 - accuracy: 0.4962 - val_loss: 1.2711 - val_accuracy: 0.5096
Epoch 7/20
225/225 [==============================] - 65s 289ms/step - loss: 1.2665 - accuracy: 0.5151 - val_loss: 1.2609 - val_ac

In [11]:
#Save model structure and model weights to file
cnn_model_json = cnn_model.to_json()
with open("cnn_model.json", "w") as json_file:
    json_file.write(cnn_model_json)
cnn_model.save_weights("cnn_model.h5")

In [12]:
#Ensure model predictions working correctly
print(cnn_model.predict(X_test))

[[2.3285492e-01 3.0285705e-02 3.3384916e-01 ... 9.1298163e-02
  3.4215279e-02 2.2027995e-01]
 [1.6427286e-01 8.1619076e-02 1.4782828e-01 ... 1.1489506e-01
  9.1522634e-02 1.4057371e-01]
 [2.5206140e-01 1.7905731e-03 1.9351982e-01 ... 3.6664400e-01
  1.8343121e-02 1.6651542e-01]
 ...
 [6.6877343e-02 5.7686048e-05 2.6649933e-02 ... 3.9973357e-01
  1.6553917e-03 4.5138955e-01]
 [3.4251115e-01 2.8320687e-04 3.9529070e-02 ... 2.6910579e-01
  1.1417701e-04 5.1717903e-02]
 [1.3911027e-01 4.1390671e-03 8.2435340e-02 ... 6.3320816e-01
  1.5848564e-04 1.4046903e-01]]


In [13]:
#Evaluate accuracy of model
cnn_model.evaluate(X_test, y_test)

225/225 [==============================] - 3s 14ms/step - loss: 1.1649 - accuracy: 0.5694


[1.1648504734039307, 0.569378674030304]